In [2]:
import pandas as pd
import numpy as np

In [3]:
movies=pd.read_csv('dataset/movies.csv')
ratings=pd.read_csv('dataset/ratings.csv')

#### Approach
1. Building a Model which recommends movies based on the ratings of similar users'.
    - We will use ratings dataframe to build this model
2. Building a Model which recommendss movies similar to those a user has liked in the past.
    - This will take the output of above modes as input and then we will build the model w.r.t. movies dataframe.
3. As a result this will create a hybrid model.

In [7]:
from sklearn.neighbors import NearestNeighbors
user_matrix=ratings.pivot(index='userId',columns='movieId',values='rating')
user_matrix.fillna(0,inplace=True)
user_matrix_array=user_matrix.values
knn=NearestNeighbors(metric='cosine',algorithm='brute')
knn.fit(user_matrix_array)

NearestNeighbors(algorithm='brute', metric='cosine')

In [14]:
user_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
def top_recommendations(userId):
    user_index = user_matrix.index.get_loc(userId)
    distances, indices = knn.kneighbors([user_matrix_array[user_index]], n_neighbors=6)
    similar_user_indices = indices.flatten()[1:]
    
    top_movies = []
    for user in similar_user_indices:
        similar_user_ratings = user_matrix.iloc[user]
        top_movies_user = similar_user_ratings.sort_values(ascending=False).head(5).index
        top_movies.extend(top_movies_user)
        
    top_movies_unique = list(set(top_movies))[:20]
    
    return top_movies_unique

def final_recommendations(userId):
    movie_indices = top_recommendations(userId)
    
    user_ratings = ratings[ratings['userId'] == userId]
    liked_movies = user_ratings[user_ratings['rating'] >= 4]['movieId'].tolist()
    
    movie_indices_in_matrix = [movie for movie in movie_indices if movie in user_matrix.index]

    item_item_matrix = user_matrix.loc[movie_indices_in_matrix]
    
    item_item_matrix_array = item_item_matrix.values
    
    movie_knn = NearestNeighbors(metric='cosine', algorithm='brute')
    movie_knn.fit(item_item_matrix_array)
    
    similar_movies = []
    for liked_movie in liked_movies:
        if liked_movie in item_item_matrix.index:
            movie_index = item_item_matrix.index.get_loc(liked_movie)
            distances, indices = movie_knn.kneighbors([item_item_matrix_array[movie_index]], n_neighbors=6)
            similar_movie_indices = indices.flatten()[1:]  # Exclude the movie itself
            similar_movies.extend(item_item_matrix.index[similar_movie_indices])
    
    final_recommendations = list(set(similar_movies))[:5]
    
    final_movies_info = movies[movies['movieId'].isin(final_recommendations)]
    
    return final_movies_info[['title', 'avg_movie_rating']]

# Example usage:
user_id = 1 
final_top_5_movies = final_recommendations(user_id)

# Display the top 5 final movie recommendations
print("Top 5 final movie recommendations:")
print(final_top_5_movies)


Top 5 final movie recommendations:
                title  avg_movie_rating
0           Toy Story          3.920930
257      Pulp Fiction          4.197068
398     Fugitive, The          3.992105
418     Jurassic Park          3.750000
461  Schindler's List          4.225000
